In [1]:
""" Generates in-house CRC/BM B3 Report for cross-checking with system's Report """

import numpy as np
import pandas as pd
import sqlalchemy as sa
import mysql.connector
import login

In [2]:
qry = "select m.massis_type_description, m.display_contractor_description, current_expired, contract, \
right(e.sor_version,3) as edition, start_date, m.anticipated_end_date, period, contract_score \
from ia_contract_summaries \
left join mass_contracts as m using (contract) \
left join erp.contracts as e using (contract) \
where current_expired is not null"

try:
    engine = sa.create_engine('mysql+mysqlconnector://{}:{}@{}/flbwmass'.format(login.username, login.password, login.mysql_host))
    rs = pd.read_sql(qry, con=engine, parse_dates=['start_date', 'anticipated_end_date'])
except mysql.connector.Error as e:
    print(e)
finally:
    engine.dispose()

In [3]:
def increment_period(period, quarter_step):
    start_year = int(period[:4])
    start_quarter = int(period[-1:])
    stacked_quarter = start_quarter + quarter_step
    year_step = (stacked_quarter - 1) // 4
    remainder_quarter = (stacked_quarter) % 4 if (stacked_quarter) % 4 != 0 else 4
    return str(start_year + year_step) + 'Q' + str(remainder_quarter)

def grant_probation(start_period=None, edition='39'):
    return increment_period(start_period, 4) if edition >= '70' else increment_period(start_period, 5)

def grant_probation_70(start_period=None):
    return increment_period(start_period, 4)

In [4]:
reporting_period = '2017Q3'
reporting_cutoff = increment_period(reporting_period, -13)

df = rs[(rs['massis_type_description'] == 'DTC') & (rs['current_expired'] == 'Current Contract') & (rs['period'] <= reporting_period)].copy()
df['last_current_period'] = df.groupby('contract')['period'].transform(lambda x: x.max())
df.loc[df['edition'].str.startswith('0'), 'edition'] = df['edition'].str[1:]
df['start_date'] = df['start_date'].dt.strftime('%#d-%b-%Y')
df['anticipated_end_date'] = df['anticipated_end_date'].dt.strftime('%#d-%b-%Y')
df = df[df['last_current_period'] >= reporting_cutoff]
df.drop(['massis_type_description', 'current_expired','last_current_period'], axis=1, inplace=True)
df.set_index(['display_contractor_description','contract','edition','start_date','anticipated_end_date','period'], inplace=True)

In [5]:
pv = df.unstack()
idx = pd.IndexSlice

pv['probation_start'] = list(zip(*pv.notnull().idxmax(axis=1)))[1]
pv['probation_end'] = np.where(pv.index.get_level_values('edition') >= '70', \
                                pv['probation_start'].apply(lambda t: grant_probation_70(t)), pv['probation_start'].apply(lambda t: grant_probation(t)))
pv['score_gte_60'] = np.where(pv['probation_end'] > reporting_period, None, \
                              np.where(np.min(pv, axis=1) >= 60, 'Yes', 'No'))
avg = []
for i, (start, e) in enumerate(zip(pv['probation_start'], pv['probation_end'])):
    end = reporting_period if e > reporting_period else e
    avg.append(np.mean(pv.loc[:, idx['contract_score', start:end]].values, axis=1)[i])
pv['average'] = np.where(pv['probation_end'] > reporting_period, None, avg)
pv['eligible'] = np.where(pv['probation_end'] > reporting_period, None, \
                          np.where((pv['score_gte_60'] == 'No') | (pv['average'] < 65), 'Not Eligible', 'Eligible'))
pv.iloc[:, pv.columns.get_level_values(0) == 'average'] = pv.iloc[:, pv.columns.get_level_values(0) == 'average'].astype('float')

pv.insert(0, 'item', np.arange(len(pv))+1)
pv.set_index('item', append=True, inplace=True)
pv = pv.reorder_levels(['display_contractor_description', 'item', 'contract', 'edition', 'start_date', 'anticipated_end_date'])
pv.index.names = ['Contractor Name', 'Item', 'Contract No.', 'DTC Edition', 'Commencement', 'Completion']
pv.columns.names=[None, None]

probation_mask = pv.copy()
pv.drop(['probation_start', 'probation_end'], axis=1, level=0, inplace=True)
pv.columns.set_levels(['Quaterly MASS Score and Performance','Probation Start', 'Probation End','Score >= 60','Average Score','Eligible'], level=0, inplace=True)

pv = pv.round(2)
pv.iloc[:, pv.columns.get_level_values(0) == 'average'] = \
pv.iloc[:, pv.columns.get_level_values(0) == 'average'].apply(lambda x: pd.Series.round(x, 2))

pv = pv.iloc[:, -13:]

In [6]:
# probation_mask['contract_score'] = False
probation_mask.ix[probation_mask.iloc[:, -6].isnull(), :-5] = 'gray'
probation_mask.ix[probation_mask.iloc[:, -6].isnull(), :-5] = False
for i, (start, e) in enumerate(zip(probation_mask['probation_start'], probation_mask['probation_end'])):
    end = reporting_period if e > reporting_period else e
    ridx = probation_mask.iloc[i].name
    probation_mask.loc[idx[ridx[0], ridx[1], ridx[2], ridx[3], ridx[4]], idx['contract_score', start:end]] = 'yellow'

probation_mask.drop(['Probation Start', 'Probation End'], axis=1, level=0, inplace=True)
probation_mask.columns.set_levels(['Quaterly MASS Score and Performance','Probation Start', 'Probation End','Score >= 60','Average Score','Eligible'], level=0, inplace=True)

D:\Continuum\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [7]:
def color_average(val):
    return np.where(pd.isnull(val), 'color: black', np.where(val.astype('float') >= 65, 'color: #3567B3', 'color: red'))

def color_score(val):
    return np.where(pd.isnull(val), 'color: black', np.where(val.astype('float') >= 60, 'color: #3567B3', 'color: red'))

# def color_expired(val):
#     return ['background-color: #e6e6e6' if pd.isnull(val[9]) else 'background-color: white']

def legend(val):
    return pd.DataFrame(np.where(probation_mask.iloc[:, -15:-2] == 'yellow', \
                                 'background-color: #FFE800', np.where(probation_mask.iloc[:, -15:-2] == 'gray', 
                                                                       'background-color: #DCDCDC', 'background-color: white')), index=val.index, columns=val.columns)

s = pv.style.apply(color_average, subset='Average Score')
s.apply(color_score, subset='Quaterly MASS Score and Performance')
# s.apply(color_expired, subset=['Quaterly MASS Score and Performance', 'Score >= 60', 'Average Score', 'Eligible'], axis=1)
s.apply(legend, axis=None)
# s.to_excel(reporting_period + '-B3-Verification.xlsx', engine='openpyxl')
with open('./output/' + reporting_period + '-B3-Data-Checking.html', 'w') as html:
    html.write(s.set_table_attributes('border="1" cellpadding="4" style="margin: 10px auto; border-collapse: collapse; font-family: Arial; font-size: 95%"')\
               .render().replace('nan','').replace('None',''))